# Parasite Genotype-Specific Protection in HbAS Carriers

This notebook reproduces key protective efficacy contrasts reported by Band et al. (2022) using synthetic data for visualization. It emphasizes the genotype-specific differences between Pfsa+ and Pfsa− parasite strains and illustrates linkage disequilibrium patterns among Pfsa loci.

## Heterozygous Advantage Mechanisms

HbAS (sickle cell trait) confers protection through altered erythrocyte biomechanics, reduced cytoadherence/rosetting, and enhanced immune clearance. The Band et al. findings refine this narrative by showing parasite genotypes can erode or amplify these effects, producing a conditional heterozygote advantage.

In [ ]:
import json
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px

sns.set_theme(style="white", font="serif")
plt.rcParams.update({
    "axes.edgecolor": "black",
    "axes.linewidth": 0.8,
    "axes.labelsize": 12,
    "axes.titlesize": 13,
    "figure.dpi": 120,
})

stats = json.loads(Path("../data/key_statistics.json").read_text())


In [ ]:
rr_data = pd.DataFrame({
    "Genotype": ["Pfsa+", "Pfsa-"],
    "RR": [stats["risk_ratios"]["pfsa_plus"]["rr"], stats["risk_ratios"]["pfsa_minus"]["rr"]],
    "CI_low": [stats["risk_ratios"]["pfsa_plus"]["ci_low"], stats["risk_ratios"]["pfsa_minus"]["ci_low"]],
    "CI_high": [stats["risk_ratios"]["pfsa_plus"]["ci_high"], stats["risk_ratios"]["pfsa_minus"]["ci_high"]],
})
rr_data


## Protective Efficacy Comparison

The contrast between Pfsa+ and Pfsa− infections illustrates the genotype-specific nature of HbAS protection.

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
sns.barplot(data=rr_data, x="Genotype", y="RR", palette=["#4C72B0", "#55A868"], ax=ax)
ax.set_ylabel("Relative Risk (HbAS vs HbAA)")
ax.set_xlabel("Parasite Genotype")
ax.set_title("Genotype-Specific Protection")
ax.axhline(1.0, linestyle="--", color="black", linewidth=0.8)
sns.despine()
plt.show()


## Forest Plot of Confidence Intervals

A forest plot highlights the stark difference in effect sizes between Pfsa+ and Pfsa− infections.

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
y_positions = np.arange(len(rr_data))
ax.errorbar(
    rr_data["RR"],
    y_positions,
    xerr=[rr_data["RR"] - rr_data["CI_low"], rr_data["CI_high"] - rr_data["RR"]],
    fmt="o",
    color="black",
    ecolor="#4C72B0",
    capsize=4,
)
ax.set_yticks(y_positions)
ax.set_yticklabels(rr_data["Genotype"])
ax.set_xlabel("Relative Risk (HbAS vs HbAA)")
ax.set_title("Forest Plot: Pfsa Genotype Effects")
ax.axvline(1.0, linestyle="--", color="gray", linewidth=0.8)
sns.despine()
plt.show()


## Linkage Disequilibrium (LD) Concept Map

This synthetic heatmap captures the idea of elevated LD among Pfsa loci, contrary to typical expectations in *P. falciparum*.

In [ ]:
ld_matrix = np.array([
    [1.0, 0.72, 0.65],
    [0.72, 1.0, 0.58],
    [0.65, 0.58, 1.0],
])
fig, ax = plt.subplots(figsize=(4, 4))
sns.heatmap(ld_matrix, annot=True, cmap="Greys", vmin=0, vmax=1, square=True, cbar=False, ax=ax)
ax.set_xticklabels(["Pfsa1", "Pfsa2", "Pfsa3"], rotation=45, ha="right")
ax.set_yticklabels(["Pfsa1", "Pfsa2", "Pfsa3"], rotation=0)
ax.set_title("Synthetic LD Map")
plt.show()


## Interactive Plotly Visualization

An interactive alternative for exploring relative risk differences.

In [ ]:
fig = px.scatter(
    rr_data,
    x="Genotype",
    y="RR",
    error_y=rr_data["CI_high"] - rr_data["RR"],
    error_y_minus=rr_data["RR"] - rr_data["CI_low"],
    title="Interactive Risk Ratio Comparison",
    template="simple_white",
)
fig.update_traces(marker=dict(size=12, color="#4C72B0"))
fig.update_layout(font=dict(family="Times New Roman", size=14))
fig.show()
